In [ ]:
import os                   # 운영체제와 상호작용하기 위한 라이브러리. 여기서는 폴더 생성에 사용됩니다.
import base64               # Base64 형식의 데이터를 다루기 위한 라이브러리. OpenAI API가 이미지를 이 형식으로 반환합니다.
from dotenv import load_dotenv # .env 파일에서 환경 변수(API 키 등)를 로드하기 위한 라이브러리입니다.
from openai import OpenAI    # OpenAI의 API를 사용하기 위한 라이브러리입니다.

import cv2                  # OpenCV 라이브러리. 이미지 처리(필터 적용, 저장 등)를 위해 사용됩니다.
import numpy as np          # 수치 계산, 특히 행렬 연산을 위한 라이브. OpenCV 이미지는 NumPy 배열 형태로 다뤄집니다.

# --- 필터 설정 값 ---
# 여기 있는 값들을 수정해서 필터의 강도를 쉽게 조절할 수 있습니다.
# 유지보수 및 설정 변경의 용이성을 위해 설정 값들을 코드 상단에 변수로 빼두었습니다.

# "더 하얗게" 필터의 강도 조절 값입니다. 양수 값이며, 값이 클수록 이미지가 더 밝아집니다. (예: 10, 30, 50)
BRIGHTNESS_ADJUSTMENT: int = 40

# "더 뿌옇게" 필터의 강도 조절 값입니다. 반드시 '홀수'여야 합니다. 값이 클수록 더 많이 흐려집니다. (예: 9, 21, 35)
BLUR_KERNEL_SIZE: int = 21


# --- 함수 정의 ---

def generate_image_data(client: OpenAI, prompt: str) -> bytes:
    """
    OpenAI DALL-E API를 사용하여 프롬프트를 기반으로 이미지를 생성하고, 생성된 이미지 데이터를 반환합니다.

    Args:
        client (OpenAI): OpenAI API와 통신하기 위한 클라이언트 객체입니다.
        prompt (str): 이미지 생성을 위한 텍스트 설명입니다. 이 텍스트를 기반으로 LLM이 이미지를 그립니다.

    Returns:
        bytes: 생성된 이미지의 원본 데이터 (base64로 디코딩된 바이트 형태). 이 데이터는 OpenCV에서 사용하기 전 단계입니다.
    """
    print(f"'{prompt}' 프롬프트로 이미지 생성을 시작합니다...")
    
    # OpenAI의 이미지 생성 API를 호출합니다.
    response = client.images.generate(
        model="dall-e-3",      # 사용할 이미지 생성 모델의 이름입니다.
        prompt=prompt,         # 이미지 생성을 위한 텍스트 설명입니다.
        size="1024x1024",      # 생성할 이미지의 크기입니다.
        quality="standard",    # 이미지의 품질 (standard 또는 hd)
        n=1,                   # 한 번의 요청으로 생성할 이미지의 개수입니다.
        response_format="b64_json" # 응답 형식을 b64_json으로 지정하여 이미지를 base64 문자열로 받습니다.
    )
    
    # 응답 데이터에서 base64로 인코딩된 이미지 문자열을 추출합니다.
    image_base64 = response.data[0].b64_json
    
    # base64 문자열을 디코딩하여 순수한 이미지 바이트 데이터로 변환합니다.
    image_bytes = base64.b64decode(image_base64)
    
    print("이미지 생성이 완료되었습니다.")
    return image_bytes


def apply_brightness_filter(image: np.ndarray, value: int) -> np.ndarray:
    """
    OpenCV를 사용하여 이미지의 밝기를 조절합니다. ("더 하얗게")

    Args:
        image (np.ndarray): 밝기를 조절할 원본 이미지 객체 (NumPy 배열 형태).
        value (int): 밝기 조절 값. 양수이면 밝아지고, 음수이면 어두워집니다.

    Returns:
        np.ndarray: 밝기가 조절된 새로운 이미지 객체 (NumPy 배열 형태).
    """
    # cv2.convertScaleAbs 함수는 이미지의 대비(alpha)와 밝기(beta)를 조절합니다.
    # 여기서는 대비(alpha)는 1로 유지하고 밝기(beta)만 조절하여 이미지를 전체적으로 밝게 만듭니다.
    # 이 함수는 연산 결과가 픽셀 값의 범위(0-255)를 벗어나면 자동으로 조절해주는 장점이 있습니다.
    brightened_image = cv2.convertScaleAbs(image, alpha=1.0, beta=value)
    return brightened_image


def apply_blur_filter(image: np.ndarray, kernel_size: int) -> np.ndarray:
    """
    OpenCV를 사용하여 이미지에 가우시안 블러(흐림 효과)를 적용합니다. ("더 뿌옇게")

    Args:
        image (np.ndarray): 블러를 적용할 원본 이미지 객체 (NumPy 배열 형태).
        kernel_size (int): 블러 효과의 강도를 결정하는 커널 크기. 반드시 양의 홀수여야 합니다.

    Returns:
        np.ndarray: 블러가 적용된 새로운 이미지 객체 (NumPy 배열 형태).
    """
    # cv2.GaussianBlur 함수는 이미지에 가우시안 블러 효과를 적용하여 부드럽게 만듭니다.
    # (kernel_size, kernel_size)는 블러를 계산할 주변 픽셀의 범위를 나타냅니다. 이 값이 클수록 더 흐려집니다.
    # 0은 표준 편차를 의미하며, 0으로 설정하면 커널 크기에 맞춰 자동으로 계산됩니다.
    blurred_image = cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)
    return blurred_image


def generate_and_filter_image():
    """
    전체 프로세스를 실행하는 메인 함수.
    이미지 생성, 필터 적용, 그리고 파일 저장을 순서대로 진행합니다.
    """
    # .env 파일에서 환경 변수(API KEY)를 로드합니다.
    load_dotenv()
    
    # OpenAI API 클라이언트를 초기화합니다.
    client = OpenAI()

    # 이미지를 생성할 프롬프트를 정의합니다.
    prompt = "A cute cat wearing a wizard hat, digital art style"

    # 1. LLM을 통해 이미지 데이터 생성
    image_bytes = generate_image_data(client, prompt)

    # 2. 이미지 데이터를 OpenCV가 다룰 수 있는 형태로 변환
    #   - np.frombuffer를 사용해 바이트 데이터를 NumPy 배열로 변환합니다.
    #   - cv2.imdecode를 사용해 NumPy 배열을 이미지 객체(행렬)로 디코딩합니다.
    image_array = np.frombuffer(image_bytes, np.uint8)
    original_image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

    # 3. 필터 적용
    print("필터 적용을 시작합니다...")
    # "더 하얗게" 필터 적용
    bright_image = apply_brightness_filter(original_image, BRIGHTNESS_ADJUSTMENT)
    
    # "더 뿌옇게" 필터 적용
    blur_image = apply_blur_filter(original_image, BLUR_KERNEL_SIZE)
    print("필터 적용이 완료되었습니다.")

    # 4. 이미지 파일로 저장
    #   - 결과물을 저장할 폴더가 없으면 생성합니다.
    output_dir = "../backend/services/letter_image/filtered_emotion_letter.jpg"
    
    os.makedirs(output_dir, exist_ok=True)

    #   - cv2.imwrite 함수를 사용하여 이미지 객체를 파일로 저장합니다.
    cv2.imwrite(os.path.join(output_dir, "original_image.png"), original_image)
    cv2.imwrite(os.path.join(output_dir, "bright_image.png"), bright_image)
    cv2.imwrite(os.path.join(output_dir, "blur_image.png"), blur_image)

    print(f"생성된 이미지들이 '{output_dir}' 폴더에 성공적으로 저장되었습니다.")


# --- 프로그램 실행 ---

# 이 스크립트 파일이 직접 실행될 때만 아래의 코드를 실행하라는 의미입니다.
# 다른 파일에서 이 파일을 import해서 사용할 경우에는 아래 코드가 실행되지 않아, 원치 않는 동작을 방지합니다.
if __name__ == "__main__":
    generate_and_filter_image()